In [1]:
# constructs level2 summary file from level 1b file using manual ifcb data
# requires: level_1b.csv, resolved_manual.csv, volumes.csv, geographic_subset.csv
import pandas as pd
import numpy as np

In [2]:
# read in level 1b file
columns = ['permalink', 'namespace_manual', 'worms_higher_order_manual', 'identification_manual', 'Biovolume']
# initialize data frame from input
samples = pd.read_csv("level_1b.csv", usecols=columns)

/Users/Kathy/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# read in taxonomic information
man_taxon_info = pd.read_csv('resolved_manual.csv', 
                                usecols=['name', 'resolved_names', 'alt_datasource'])
# merge to get taxa data
samples = pd.merge(samples, man_taxon_info, how='left', left_on='namespace_manual', right_on='name')

In [4]:
# separate out roi id from permalink
samples['roi'] = samples['permalink']
samples.roi = samples.roi.str.slice(68, 74)
# gets rid of leading zeros
samples.roi = samples.roi.str.lstrip("0")
# cut permalink to just be permalink of sample
samples.permalink = samples.permalink.str.slice(0, 67)

In [5]:
# read in volume data
volumes = pd.read_csv("volumes.csv")
# merge with samples
samples = pd.merge(samples, volumes, how='left', on='permalink')

In [6]:
# read in geolocation data
geo_data = pd.read_csv("geographic_subset.csv", usecols=['gps_furuno_latitude', 'gps_furuno_longitude', 
                                                         'date', 'pid'])
samples = pd.merge(samples, geo_data, how='left', left_on='permalink', right_on='pid')

In [7]:
# group samples into summary format
level_2 = samples
level_2 = level_2.groupby(['permalink', 'namespace_manual']).agg(
    {
        'identification_manual': 'first',
        'worms_higher_order_manual': 'first',
        'roi': 'count',
        'Biovolume': 'sum',
        'volume_imaged': 'first',
        'date': 'first',
        'gps_furuno_latitude': 'first',
        'gps_furuno_longitude': 'first'
    }
).reset_index()
# rename roi to be abundance
level_2.rename(columns={'roi': 'Abundance'}, inplace=True)

In [10]:
# save output to csv file
level_2.to_csv('level_2.csv', index=None, header=True)